In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from bs4 import BeautifulSoup as bs
from secrets import token1 

In [2]:
secret = token1

In [2]:
#scrape webpage

def get_spy():
    
    url = 'https://www.slickcharts.com/sp500'

    request = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})

    soup = bs(request.text, "lxml")

    stats = soup.find('table',class_='table table-hover table-borderless table-sm')

    df =pd.read_html(str(stats))[0]

    df['% Chg'] = df['% Chg'].str.strip('()-%')

    df['% Chg'] = pd.to_numeric(df['% Chg'])

    df['Chg'] = pd.to_numeric(df['Chg'])

    return df


In [3]:
stocks = pd.DataFrame(get_spy().set_index('#')['Symbol'])
stocks.rename({'Symbol':'Ticker'}, axis=1, inplace=True)

stocks.head(5)

,Ticker
#,
1,AAPL
2,MSFT
3,AMZN
4,GOOGL
5,GOOG


symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={secret}'
data = requests.get(api_url).json()

data['latestPrice']

In [5]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe


final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={secret}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [6]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    #print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={secret}'
    data = requests.get(batch_api_call_url).json()  
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
final_dataframe['Market Capitalization'] = final_dataframe['Market Capitalization']/(10**9) 

C:\Users\zvern\AppData\Local\Temp\ipykernel_6864\1904129517.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_6864\1904129517.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_6864\1904129517.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\Local\Temp\ipykernel_6864\1904129517.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\zvern\AppData\L

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,141.66,2292792740460,N/A
1,MSFT,267.7,2002137170240,N/A
2,AMZN,116.46,1184864040000,N/A
3,GOOGL,2359.5,1600668417992,N/A
4,GOOG,2370.76,1600668418926,N/A
...,...,...,...,...
195,DLTR,157.21,35302454420,N/A
196,PRU,97.41,36528750000,N/A
197,MNST,94.58,50096321674,N/A
198,MSCI,432.06,35112612763,N/A


In [11]:
portfolio_size = input('Enter the value your portfolio')

try:
    val = float(portfolio_size)    
except ValueError:
    print('Please enter an number \nPlease try again')
    portfolio_size = input('Enter the value your portfolio')
    val = float(portfolio_size)
    

Enter the value your portfolio 


Please enter an number 
Please try again


Enter the value your portfolio 1234233


In [12]:
position_size = val/len(final_dataframe.index)
position_size

6171.165

In [13]:
for i in range(0, len(final_dataframe.index)):
    final_dataframe.iloc[i, 3] = math.floor(position_size/(final_dataframe.iloc[i,1]))
    
    
    

In [14]:
final_dataframe.head(3)

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,141.66,2292792740460,43
1,MSFT,267.7,2002137170240,23
2,AMZN,116.46,1184864040000,52


In [15]:

writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [16]:

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

In [17]:

column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [18]:

writer.save()